<a href="https://colab.research.google.com/github/melkatewabe10/Machine-learning_LST-Estimation-/blob/main/Monthly_Ta_Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import laibrary

In [ ]:
import ee
import folium
import matplotlib.pyplot as plt
import geemap
import math
# Authenticate to Earth Engine. This will open a browser window.
ee.Authenticate()

# Initialize the Earth Engine API.
ee.Initialize(project='ee-mftewabe')

# ALB

In [ ]:
# -------------------------------------------------------------------
# Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# Optional: Load ESA WorldCover and create a land mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
mask = worldcover.neq(50).And(worldcover.neq(80))  # Optional masking

# -------------------------------------------------------------------
# Loop through each year and month
# -------------------------------------------------------------------
for year in range(2021, 2025):
    print(f"Processing year: {year}")

    for month in range(1, 13):
        # Format month string as '01', '02', ..., '12'
        month_str = str(month).zfill(2)

        # Get the first day of the month
        start_date = ee.Date(f"{year}-{month_str}-01")

        # Get the last day of the month using advance()
        end_date = start_date.advance(1, 'month').advance(-1, 'day')

        print(f"  Processing {year}-{month_str}")

        # Get monthly mean ALBEDO
        monthly_albedo = (ee.ImageCollection("MODIS/061/MCD43A3")
                          .filterDate(start_date, end_date)
                          .filterBounds(taiwan)
                          .select('Albedo_WSA_Band1')
                          .mean()
                          .clip(taiwan))

        # Scale the values (MODIS Albedo scale factor = 0.001)
        albedo_scaled = monthly_albedo.multiply(0.001)

        # Optional masking
        albedo_masked = albedo_scaled.updateMask(mask)

        # Export task to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=albedo_masked,
            description=f'ALB_{year}_{month_str}',
            folder='MMASK_FIVE',
            fileNamePrefix=f'ALB_{year}_{month_str}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"  Export task for {year}-{month_str} started.")


# DSR

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# Optional: Load ESA WorldCover and create a land mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
mask = worldcover.neq(50).And(worldcover.neq(80))  # Optional: remove built-up and water

# -------------------------------------------------------------------
# 2. Loop through each year and month
# -------------------------------------------------------------------
for year in range(2021, 2025):
    print(f"Processing year: {year}")

    for month in range(1, 13):
        # Format month string as two digits
        month_str = str(month).zfill(2)

        # Get start and end dates of the month
        start_date = ee.Date(f"{year}-{month_str}-01")
        end_date = start_date.advance(1, 'month').advance(-1, 'day')

        print(f"  Processing {year}-{month_str}")

        # Compute the monthly mean DSR
        DSR = (ee.ImageCollection("MODIS/062/MCD18A1")
               .filterDate(start_date, end_date)
               .filterBounds(taiwan)
               .select('GMT_0600_DSR')
               .mean()
               .clip(taiwan))

        # Optional: Apply land mask
        DSR_masked = DSR.updateMask(mask)

        # Export to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=DSR_masked,
            description=f'DSR_{year}_{month_str}',
            folder='MMASK_FIVE',
            fileNamePrefix=f'DSR_{year}_{month_str}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year}-{month_str} started.")


In [ ]:
!pip install rasterio

In [ ]:

import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Path to the downloaded GeoTIFF file.
filename = '/content/drive/MyDrive/MMASK_FIVE/NDVI_2022_08.tif'

# Open the file with rasterio
with rasterio.open(filename) as src:
    # Read the first band (assuming the LST is stored in a single band)
    lst_data = src.read(1)
    # Get metadata information
    meta = src.meta
    print("Metadata:")
    for key, value in meta.items():
        print(f"{key}: {value}")

# Mask no-data values (if present)
ndvi_masked = np.ma.masked_where(lst_data == src.nodata, lst_data)

# Plot the LST image using matplotlib
plt.figure(figsize=(10, 8))
cmap = plt.cm.jet  # choose a colormap
img = plt.imshow(ndvi_masked, cmap=cmap)
plt.colorbar(img, label='LST (K)')
plt.title('MODIS LST (Masked) in Kelvin')
plt.xlabel('Column #')
plt.ylabel('Row #')
plt.show()

In [ ]:
# prompt: histogram of # Path to the downloaded GeoTIFF file.
# filename = '/content/drive/MyDrive/MONTHLY_ONE/NDWI_2002_07.tif'

import matplotlib.pyplot as plt
import rasterio
import numpy as np

# Path to the downloaded GeoTIFF file.
filename = '/content/drive/MyDrive/MMASK_ONE/ALB_2001_10.tif'

# Open the file with rasterio
with rasterio.open(filename) as src:
    # Read the first band
    band_data = src.read(1)

    # Get the nodata value
    nodata_value = src.nodata

# Mask no-data values
masked_data = np.ma.masked_where(band_data == nodata_value, band_data)

# Create the histogram
plt.figure(figsize=(10, 6))
plt.hist(masked_data.compressed(), bins=50)  # Use compressed() to exclude masked values
plt.xlabel('Pixel Values')
plt.ylabel('Frequency')
plt.title('Histogram of NDWI Values')
plt.show()


LAI

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Optional: Load ESA WorldCover and create a land mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
mask = worldcover.neq(50).And(worldcover.neq(80))  # Optional: remove built-up and water

# -------------------------------------------------------------------
# 3. Loop through each year and month
# -------------------------------------------------------------------
for year in range(2000, 2006):
    print(f"Processing year: {year}")

    for month in range(1, 13):
        month_str = str(month).zfill(2)
        print(f"  Processing {year}-{month_str}")

        # Define date range
        start_date = ee.Date(f"{year}-{month_str}-01")
        end_date = start_date.advance(1, 'month').advance(-1, 'day')

        # Compute monthly mean LAI
        LAI = (ee.ImageCollection("MODIS/061/MOD15A2H")
               .filterDate(start_date, end_date)
               .filterBounds(taiwan)
               .select('Lai_500m')
               .mean()
               .clip(taiwan))

        # Scale and clamp LAI values (MODIS scale factor = 0.1)
        LAI_scaled = LAI.multiply(0.1).clamp(0, 7)

        # Optional land mask
        LAI_masked = LAI_scaled.updateMask(mask)

        # Export task
        task = ee.batch.Export.image.toDrive(
            image=LAI_masked,
            description=f'LAI_{year}_{month_str}',
            folder='MMASK_ONE',
            fileNamePrefix=f'LAI_{year}_{month_str}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year}-{month_str} started.")


# LST

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Optional: Load ESA WorldCover and create a land mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
mask = worldcover.neq(50).And(worldcover.neq(80))  # Optional mask for land

# -------------------------------------------------------------------
# 3. Loop through years and months
# -------------------------------------------------------------------
for year in range(2021, 2025):  # Adjust range as needed
    print(f"Processing year: {year}")

    for month in range(1, 13):
        month_str = str(month).zfill(2)
        print(f"  Processing {year}-{month_str}")

        # Define date range
        start_date = ee.Date(f"{year}-{month_str}-01")
        end_date = start_date.advance(1, 'month').advance(-1, 'day')  # End of month

        # Retrieve and process LST
        LST = (ee.ImageCollection("MODIS/061/MOD11A1")
               .filterDate(start_date, end_date)
               .filterBounds(taiwan)
               .select('LST_Day_1km')
               .mean()
               .clip(taiwan))

        # Scale to Kelvin (MODIS LST scale factor is 0.02)
        LST_scaled = LST.multiply(0.02)

        # Optional: Apply land mask
        LST_masked = LST_scaled.updateMask(mask)

        # Export to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=LST_masked,
            description=f'LST_{year}_{month_str}',
            folder='MMASK_FIVE',  # Customize the destination folder
            fileNamePrefix=f'LST_{year}_{month_str}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year}-{month_str} started.")


# ENVI_NDVI

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Optional: Load ESA WorldCover and create a mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
mask = worldcover.neq(50).And(worldcover.neq(80))  # Optional: remove built-up and water

# -------------------------------------------------------------------
# 3. Loop through each year and each month
# -------------------------------------------------------------------
for year in range(2021, 2025):  # Adjust range as needed
    print(f"Processing year: {year}")

    for month in range(1, 13):
        month_str = str(month).zfill(2)
        print(f"  Processing {year}-{month_str}")

        # Define monthly date range
        start_date = ee.Date(f"{year}-{month_str}-01")
        end_date = start_date.advance(1, 'month').advance(-1, 'day')

        # Compute monthly mean EVI
        EVI = (ee.ImageCollection("MODIS/061/MOD13A2")
               .filterDate(start_date, end_date)
               .filterBounds(taiwan)
               .select('EVI')
               .mean()
               .clip(taiwan))

        # Apply MODIS scale factor (0.0001)
        EVI_scaled = EVI.multiply(0.0001)

        # Optional: apply land cover mask
        EVI_masked = EVI_scaled.updateMask(mask)

        # Export to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=EVI_masked,
            description=f'EVI_{year}_{month_str}',
            folder='MMASK_FIVE',  # Customize folder name as needed
            fileNamePrefix=f'EVI_{year}_{month_str}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,  # MODIS resolution (~1 km)
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year}-{month_str} started.")


# NDWI

In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Define NDWI computation function
# -------------------------------------------------------------------
def compute_ndwi(image):
    # Select NIR and SWIR bands, scale them, and compute NDWI
    nir = image.select('sur_refl_b02').multiply(0.0001)
    swir = image.select('sur_refl_b07').multiply(0.0001)
    ndwi = nir.subtract(swir).divide(nir.add(swir)).rename('NDWI')
    return image.addBands(ndwi)

# -------------------------------------------------------------------
# 3. Mask NDWI values outside the range [-0.5, 1]
# -------------------------------------------------------------------
def mask_ndwi(image):
    # Get the NDWI band
    ndwi = image.select('NDWI')
    # Apply mask for NDWI between -0.5 and 1
    masked_ndwi = ndwi.updateMask(ndwi.gte(-0.5).And(ndwi.lte(1)))
    return image.addBands(masked_ndwi)

# -------------------------------------------------------------------
# 4. Loop through each year and month, export NDWI
# -------------------------------------------------------------------
for year in range(2021, 2022):  # MYD09A1 starts from mid-2002
    print(f"Processing year: {year}")

    for month in range(1, 13):
        month_str = str(month).zfill(2)
        print(f"  Processing {year}-{month_str}")

        # Define monthly date range
        start_date = ee.Date(f"{year}-{month_str}-01")
        end_date = start_date.advance(1, 'month').advance(-1, 'day')

        # Load MODIS Surface Reflectance data
        collection = (ee.ImageCollection('MODIS/061/MYD09A1')
                      .filterDate(start_date, end_date)
                      .filterBounds(taiwan)
                            )

        # Compute monthly mean NDWI and apply the NDWI masking
        ndwi_mean = collection.map(compute_ndwi).map(mask_ndwi).select('NDWI').mean().clip(taiwan)

        # Clip NDWI values outside the range of -0.5 to 1
        ndwi_clip = ndwi_mean.updateMask(ndwi_mean.gte(-0.5).And(ndwi_mean.lte(1)))

        # Export to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=ndwi_clip,
            description=f'NDWI_{year}_{month_str}',
            folder='DELET21',
            fileNamePrefix=f'NDWI_{year}_{month_str}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year}-{month_str} started.")


In [ ]:
# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. (Optional) Load ESA WorldCover and create a mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
mask = worldcover.neq(50).And(worldcover.neq(80))  # Remove built-up and water

# -------------------------------------------------------------------
# 3. Define cloud masking and NDWI computation functions
# -------------------------------------------------------------------
def maskClouds(image):
    qa = image.select('StateQA')
    cloudBitMask = 1 << 10  # Bit 10 indicates clouds
    cloudMask = qa.bitwiseAnd(cloudBitMask).eq(0)
    return image.updateMask(cloudMask)

def compute_ndwi(image):
    nir = image.select('sur_refl_b02').multiply(0.0001)
    swir = image.select('sur_refl_b07').multiply(0.0001)
    ndwi = nir.subtract(swir).divide(nir.add(swir)).rename('NDWI')
    return image.addBands(ndwi)

# -------------------------------------------------------------------
# 4. Loop through each year and month, export NDWI
# -------------------------------------------------------------------
for year in range(2021, 2022):  # MYD09A1 starts from mid-2002
    print(f"Processing year: {year}")

    for month in range(1, 13):
        month_str = str(month).zfill(2)
        print(f"  Processing {year}-{month_str}")

        # Define monthly date range
        start_date = ee.Date(f"{year}-{month_str}-01")
        end_date = start_date.advance(1, 'month').advance(-1, 'day')

        # Load MODIS Surface Reflectance data
        collection = (ee.ImageCollection('MODIS/061/MYD09A1')
                      .filterDate(start_date, end_date)
                      .filterBounds(taiwan)
                            )

        # # Compute monthly mean NDWI
        collectiontwo = collection.map(compute_ndwi).select('NDWI').mean().clip(taiwan)
        # # # Compute monthly mean NDWI
        # collectiontwo= collection.map(maskClouds).map(compute_ndwi)

        # Compute monthly mean NDWI
        ndwi_mean = collectiontwo.select('NDWI').mean().clip(taiwan)

        # Filter extreme values
        ndwi_clip = ndwi_mean.updateMask(ndwi_mean.gte(-0.5).And(ndwi_mean.lte(1)))

        # # Optional: apply land mask
        # ndwi_masked = ndwi_clip.updateMask(mask)

        # Export to Google Drive
        task = ee.batch.Export.image.toDrive(
            image=ndwi_clip,
            description=f'NDWI_{year}_{month_str}',
            folder='NDWI_DEMO_DD',
            fileNamePrefix=f'NDWI_{year}_{month_str}',
            region=taiwan.geometry().bounds().getInfo()['coordinates'],
            scale=1000,
            crs='EPSG:32651',
            maxPixels=1e13
        )

        task.start()
        print(f"    Export task for {year}-{month_str} started.")


In [ ]:

# Create interactive map centered on Taiwan
Map = geemap.Map(center=[23.5, 121], zoom=7)

# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Define QA masking function (MODLAND QA bits 0-1)
# -------------------------------------------------------------------
def mask_modis_qa(image):
    qa = image.select('StateQA')  # MODIS MYD09A1 QA band
    modland_qa = qa.bitwiseAnd(1)  # Extract bits 0–1
    mask = modland_qa.lte(0)  # Keep only good pixels (0 or 1)
    return image.updateMask(mask)

# -------------------------------------------------------------------
# 3. Define RGB composite function
# -------------------------------------------------------------------
def compute_rgb(image):
    red = image.select('sur_refl_b01').multiply(0.0001).rename('Red')
    green = image.select('sur_refl_b04').multiply(0.0001).rename('Green')
    blue = image.select('sur_refl_b03').multiply(0.0001).rename('Blue')
    return red.addBands([green, blue])

# -------------------------------------------------------------------
# 4. Set date range for Spring (March–May)
# -------------------------------------------------------------------
year = 2024
start_date = ee.Date(f"{year}-09-01")
end_date = ee.Date(f"{year}-11-30")

# Load, mask, compute RGB, and calculate seasonal mean
collection = (ee.ImageCollection('MODIS/061/MYD09A1')
              .filterDate(start_date, end_date)
              .filterBounds(taiwan)
              .map(mask_modis_qa)
              .map(compute_rgb))

spring_rgb_mean = collection.median().clip(taiwan)

# -------------------------------------------------------------------
# 5. Visualize Spring RGB on map
# -------------------------------------------------------------------
vis_params = {
    'bands': ['Red', 'Green', 'Blue'],
    'min': 0,
    'max': 0.3,
    'gamma': 1.4
}
Map.addLayer(spring_rgb_mean, vis_params, f'Spring RGB {year}')
Map.addLayer(taiwan, {}, 'Taiwan')
Map

# # -------------------------------------------------------------------
# # 6. Export Spring RGB to Google Drive
# # -------------------------------------------------------------------
# task = ee.batch.Export.image.toDrive(
#     image=spring_rgb_mean,
#     description=f'Spring_RGB_{year}',
#     folder='RGB_Spring',
#     fileNamePrefix=f'Spring_RGB_{year}',
#     region=taiwan.geometry().bounds().getInfo()['coordinates'],
#     scale=500,
#     crs='EPSG:32651',
#     maxPixels=1e13
# )

# task.start()
# print(f"✅ Export task for Spring_RGB_{year} started.")


In [ ]:

# Create interactive map centered on Taiwan
Map = geemap.Map(center=[23.5, 121], zoom=7)

# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Optional: Load ESA WorldCover and create a mask (optional)
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
mask = worldcover.neq(50).And(worldcover.neq(80))  # Optional: remove built-up and water

# -------------------------------------------------------------------
# 3. Define QA masking function (MODLAND QA bits 0-1)
# -------------------------------------------------------------------
def mask_modis_qa(image):
    qa = image.select('DetailedQA')  # Use MOD13A1 QC_500m band
    modland_qa = qa.bitwiseAnd(1)  # Bits 0 and 1
    mask = modland_qa.lte(0)  # Accept only values 0 or 1
    return image.updateMask(mask)

# -------------------------------------------------------------------
# 4. Define RGB composite function (Using EVI or NDVI)
# -------------------------------------------------------------------
def compute_rgb(image):
    # Using EVI band for visualization, but you can choose NDVI or others.
    evi = image.select('EVI').multiply(0.0001).rename('EVI')
    # You can also visualize NDVI if needed
    # ndvi = image.select('NDVI').multiply(0.0001).rename('NDVI')
    return evi

# -------------------------------------------------------------------
# 5. Set date and load data for one month (May 2024 in this example)
# -------------------------------------------------------------------
year = 2024
month = 3
month_str = str(month).zfill(2)

start_date = ee.Date(f"{year}-{month_str}-01")
end_date = start_date.advance(1, 'month').advance(-1, 'day')

# Load, mask, and compute the RGB composite for the specified month
collection = (ee.ImageCollection('MODIS/061/MOD13A1')
              .filterDate(start_date, end_date)
              .filterBounds(taiwan)
              .map(mask_modis_qa)
              .map(compute_rgb))

# Compute the mean for the selected month (you can also use median or another statistic)
evi_mean = collection.median().clip(taiwan)

# -------------------------------------------------------------------
# 6. Visualize the EVI for the selected month on the map
# -------------------------------------------------------------------
vis_params = {
    'bands': ['EVI'],  # Choose 'EVI' or 'NDVI' depending on the type of analysis
    'min': 0.0,
    'max': 0.8,
    'gamma': 1.4
}
Map.addLayer(evi_mean, vis_params, f'EVI {year}-{month_str}')
Map.addLayer(taiwan, {}, 'Taiwan')

# Display the interactive map
Map

# # -------------------------------------------------------------------
# # 7. Export the EVI composite for the selected month to Google Drive
# # -------------------------------------------------------------------
# task = ee.batch.Export.image.toDrive(
#     image=evi_mean,
#     description=f'EVI_{year}_{month_str}',
#     folder='EVI_Exports',  # Customize folder name as needed
#     fileNamePrefix=f'EVI_{year}_{month_str}',
#     region=taiwan.geometry().bounds().getInfo()['coordinates'],
#     scale=1000,  # MODIS resolution (~1 km)
#     crs='EPSG:32651',
#     maxPixels=1e13
# )

# task.start()
# print(f"✅ Export task for EVI {year}-{month_str} started.")


Map(center=[23.5, 121], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

In [ ]:
# Create interactive map centered on Taiwan
Map = geemap.Map(center=[23.5, 121], zoom=7)

# -------------------------------------------------------------------
# 1. Define the study region (Taiwan)
# -------------------------------------------------------------------
gaul = ee.FeatureCollection("FAO/GAUL/2015/level0")
taiwan = gaul.filter(ee.Filter.eq('ADM0_NAME', 'Taiwan'))

# -------------------------------------------------------------------
# 2. Optional: Load ESA WorldCover and create a mask
# -------------------------------------------------------------------
worldcover = ee.ImageCollection("ESA/WorldCover/v200").first().clip(taiwan)
mask = worldcover.neq(50).And(worldcover.neq(80))  # Remove built-up and water

# -------------------------------------------------------------------
# 3. Define QA masking function for MOD13A1
# -------------------------------------------------------------------
def mask_modis_qa(image):
    qa = image.select('DetailedQA')
    modland_qa = qa.bitwiseAnd(1)  # Bit 0
    mask = modland_qa.lte(0)
    return image.updateMask(mask)

# -------------------------------------------------------------------
# 4. Define EVI computation function
# -------------------------------------------------------------------
def compute_evi(image):
    evi = image.select('NDVI').multiply(0.0001).rename('EVI')
    return evi

# -------------------------------------------------------------------
# 5. Set date range for Summer (JJA) 2024
# -------------------------------------------------------------------
year = 2021
start_date = ee.Date(f"{year}-03-01")
end_date = ee.Date(f"{year}-05-01").advance(-1, 'day')  # August 31

# -------------------------------------------------------------------
# 6. Load and process MODIS EVI data
# -------------------------------------------------------------------
collection = (ee.ImageCollection('MODIS/061/MOD13A1')
              .filterDate(start_date, end_date)
              .filterBounds(taiwan)
              .map(mask_modis_qa)
              .map(compute_evi))

evi_summer = collection.median().clip(taiwan)
evi_summer_masked = evi_summer.updateMask(mask)

# -------------------------------------------------------------------
# 7. Visualize EVI for Summer on the map
# -------------------------------------------------------------------
vis_params = {
    'bands': ['EVI'],
    'min': 0.0,
    'max': 0.3,
    'palette': ['ffffff', 'd9f0a3', 'addd8e', '78c679', '31a354', '006837']
}

Map.addLayer(evi_summer , vis_params, 'EVI Summer 2024')
Map.addLayer(taiwan, {}, 'Taiwan')
Map


Map(center=[23.5, 121], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…